In [4]:
import tensorflow as tf
import numpy as np
from loadData import loadDataset
import dataAugmentation
import metrics
import MobileNet
import time
import matplotlib.pyplot as plt

# Load the dataset
x_train,y_train,x_test,y_test= loadDataset("../Data/train.csv","../Data/test.csv")


# Preprocess data
x_train,x_test = x_train / 255.0,x_test / 255.0

# Apply data augmentation to x_train
x_augmented_data=np.concatenate([
    dataAugmentation.darkening(x_train[:10000], 2),
    dataAugmentation.shift_left(x_train[10000:15000],10),
    dataAugmentation.move_up(x_train[15000:20000],14),
    dataAugmentation.move_down(x_train[20000:25000],14),
    dataAugmentation.shift_right(x_train[25000:],10)
])

x_train=np.concatenate([x_train,x_augmented_data])
y_train=np.concatenate([y_train,y_train])
# Apply data augmentation to x_test
x_test = np.concatenate([
    dataAugmentation.darkening(x_test[:2000], 1.5),
    dataAugmentation.shift_left(x_test[2000:4000],8),
    dataAugmentation.move_up(x_test[4000:6000],8),
    dataAugmentation.move_down(x_test[6000:8000],8),
    dataAugmentation.shift_right(x_test[8000:],8)
])

# Check the shapes
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)


# MobileNet
mobilenet_model=MobileNet.mobile_net(x_train.shape[1:])

# Display Mobile Net model summary
mobilenet_model.summary()



# Train MoileNet
epochs=15
batch_size=32
mobilenet_start_time=time.time()
mobilenet_model=MobileNet.train(mobilenet_model, x_train, y_train, epochs, batch_size)
mobilenet_y_pred_labels=MobileNet.test(mobilenet_model,x_test)
mobilenet_end_time=time.time()



# # Calculate evaluation metrics
mobilenet_accuracy = metrics.accuracy(y_test, mobilenet_y_pred_labels)
mobilenet_prediction = metrics.precision(y_test, mobilenet_y_pred_labels)
mobilenet_recall = metrics.recall(y_test, mobilenet_y_pred_labels)
mobilenet_f1 = metrics.f1_score(y_test, mobilenet_y_pred_labels)
mobilenet_average=metrics.average(mobilenet_accuracy,mobilenet_prediction,mobilenet_recall,mobilenet_f1)
mobilenet_running_time=metrics.running_time(mobilenet_start_time,mobilenet_end_time)



#Print Results
print(f"Accuracy:\nMobileNet-{mobilenet_accuracy} ")
print(f"Precision:\nMobileNet-{mobilenet_prediction} ")
print(f"Recall:\nMobileNet-{mobilenet_recall} ")
print(f"F1-score:\nMobileNet-{mobilenet_f1}")
print(f"Average:\nMobileNet-{mobilenet_average} ")
print(f"Running time:\n MobileNet-{mobilenet_running_time:.2f} seconds")

(64000, 28, 28, 1)
(64000,)
(10000, 28, 28, 1)
(10000,)
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_21 (Conv2D)          (None, 14, 14, 32)        320       
                                                                 
 batch_normalization_39 (Ba  (None, 14, 14, 32)        128       
 tchNormalization)                                               
                                                                 
 re_lu_39 (ReLU)             (None, 14, 14, 32)        0         
                                                                 
 depthwise_conv2d_18 (Depth  (None, 14, 14, 32)        320       
 wiseConv2D)                                                     
                                                                 
 ba